# Cross-Species DESeq2: Differential Accessibility Analysis

**Goal:** Identify human-specific chromatin accessibility changes across 6 primate species.

**Pipeline:**
1. Load quantification feather files (from `04_quantification/`)
2. Subset to cell types shared across all species
3. DESeq2 with `~species + celltype` design for PCA
4. Pairwise DESeq2: Human vs each other species
5. Heatmap of top differential peaks
6. Manhattan plots

**Environment:** `R_deseq2` (R 4.3, DESeq2 1.42, created via mamba)

**Input data:** Feather files from `cross_species_consensus/04_quantification/`
These contain integer fragment counts: peaks (rows) x cell types (columns).

In [ ]:
# =============================================================================
# Cell 1: Load packages
# =============================================================================
suppressPackageStartupMessages({
  library(DESeq2)
  library(arrow)       # read feather files
  library(ggplot2)
  library(pheatmap)
  library(ggrepel)
  library(gtools)      # mixedorder for chromosome sorting
  library(dplyr)
  library(tibble)
  library(tidyr)
  library(readr)
  library(RColorBrewer)
})
message("All packages loaded")

In [ ]:
# =============================================================================
# Cell 2: Configuration -- edit paths and parameters here
# =============================================================================

# Base directory for all data
BASE <- "/cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/peaks"
QUANT_DIR <- file.path(BASE, "cross_species_consensus/04_quantification")

# BED file with peak coordinates (chr, start, end, region_id)
BED_FILE <- file.path(BASE, "cross_species_consensus/02_merged_consensus/unified_consensus_hg38_with_ids.bed")

# Chromosome sizes for Manhattan plots
CHROMSIZES_FILE <- "~/jjans/analysis/cerebellum/genomes_new/homo_sapiens/hg38.chrom.sizes"

# Output directory
OUT_DIR <- file.path(BASE, "cross_species_consensus/05_deseq2_R")
dir.create(OUT_DIR, showWarnings = FALSE, recursive = TRUE)
dir.create(file.path(OUT_DIR, "plots"), showWarnings = FALSE)

# Species to analyze
SPECIES <- c("Human", "Bonobo", "Chimpanzee", "Gorilla", "Macaque", "Marmoset")

# Minimum total fragment count per cell type (filter low-coverage samples)
FRAG_THRESH <- 1e6

message("Output directory: ", OUT_DIR)

In [ ]:
# =============================================================================
# Cell 3: Load and merge quantification data from all species
# =============================================================================
# Each feather file has peaks as rows and cell types as columns.
# Column names are like "Adipocytes.fragments.tsv" -- we clean these up
# and prefix with species name (e.g. "Human_Adipocytes").
# NOTE: The feather data already contains integer counts (no need to
# multiply by region length like the old TSV pipeline).

all_dfs <- list()

for (sp in SPECIES) {
  fpath <- file.path(QUANT_DIR, paste0("quantification_", sp, ".feather"))
  df <- as.data.frame(read_feather(fpath))

  # Set peak IDs as row names
  rownames(df) <- df$index
  df$index <- NULL

  # Clean column names: remove ".fragments.tsv" suffix, prefix with species
  colnames(df) <- gsub(".fragments.tsv", "", colnames(df), fixed = TRUE)
  colnames(df) <- paste0(sp, "_", colnames(df))

  # Filter: keep only cell types with enough total fragments
  col_sums <- colSums(df)
  keep <- col_sums > FRAG_THRESH
  df <- df[, keep, drop = FALSE]

  message(sp, ": ", ncol(df), " cell types kept (of ", sum(!keep) + sum(keep), ")")
  all_dfs[[sp]] <- df
}

# Inner join on shared peak IDs (row names)
shared_peaks <- Reduce(intersect, lapply(all_dfs, rownames))

# unname() prevents cbind from prepending list names (e.g. "Human.Human_ECs")
all_data <- do.call(cbind, unname(lapply(all_dfs, function(x) x[shared_peaks, , drop = FALSE])))
all_data[is.na(all_data)] <- 0

message("\nMerged matrix: ", nrow(all_data), " peaks x ", ncol(all_data), " samples")

In [ ]:
# =============================================================================
# Cell 4: Build sample metadata & identify shared cell types
# =============================================================================
# Parse species and cell type from column names (format: "Species_CellType")

sample_names <- colnames(all_data)
coldata <- data.frame(
  sample   = sample_names,
  species  = sub("_.*", "", sample_names),
  celltype = sub("^[^_]+_", "", sample_names),
  row.names = sample_names,
  stringsAsFactors = FALSE
)
coldata$species  <- factor(coldata$species, levels = SPECIES)
coldata$celltype <- factor(coldata$celltype)

# Show samples per species
message("Samples per species:")
print(table(coldata$species))

# Find cell types present in ALL 6 species
ct_per_species <- split(coldata$celltype, coldata$species)
shared_ct <- Reduce(intersect, lapply(ct_per_species, as.character))
message("\nShared cell types across all ", length(SPECIES), " species: ", length(shared_ct))
cat(paste(" ", sort(shared_ct)), sep = "\n")

In [ ]:
# =============================================================================
# Cell 5: Subset to shared cell types & sanitize factor levels
# =============================================================================
# This ensures a balanced design for DESeq2 (same cell types across species).
# We also clean up special characters in cell type names (+, etc.) to avoid
# DESeq2 warnings and formula issues.

keep_samples <- coldata$celltype %in% shared_ct
all_data_shared <- all_data[, keep_samples]
coldata_shared  <- coldata[keep_samples, ]
coldata_shared$celltype <- droplevels(coldata_shared$celltype)

# Sanitize cell type names: replace '+' with 'pos', remove other non-safe chars
# DESeq2 only likes letters, numbers, '_' and '.'
clean_levels <- levels(coldata_shared$celltype)
clean_levels <- gsub("\\+", "pos", clean_levels)
clean_levels <- gsub("[^A-Za-z0-9_.]", "_", clean_levels)
levels(coldata_shared$celltype) <- clean_levels

# Also update column names in the count matrix to match
new_colnames <- paste0(coldata_shared$species, "_", coldata_shared$celltype)
colnames(all_data_shared) <- new_colnames
rownames(coldata_shared) <- new_colnames
coldata_shared$sample <- new_colnames

message("Subset to shared cell types: ", ncol(all_data_shared), " samples")
message("Cell types: ", paste(levels(coldata_shared$celltype), collapse = ", "))

In [ ]:
# =============================================================================
# Cell 6 (OPTIONAL): Subset to specific cell types of interest
# =============================================================================
# Skip this cell to use ALL shared cell types, or run it to subset to a
# curated list. Edit SELECTED_CELLTYPES below.
# NOTE: names must match the sanitized versions (+ replaced with "pos").
# Run cell 5 first to see the available cell types.

SELECTED_CELLTYPES <- c(
  "Colonocytes",
  "ECs",
  "Enterocytes",
  "Goblet_cells",
  "Naive_B_cells",
  "Plasma_B_cells",
  "Specialized_Fibroblasts_SYNMpos",
  "Stem_cells",
  "T_cells"
)

# Validate: check which requested cell types are actually available
available <- levels(coldata_shared$celltype)
missing <- setdiff(SELECTED_CELLTYPES, available)
if (length(missing) > 0) {
  warning("These cell types are NOT in the shared set and will be skipped:\n  ",
          paste(missing, collapse = ", "),
          "\n\nAvailable cell types:\n  ",
          paste(available, collapse = ", "))
}

# Subset
keep <- coldata_shared$celltype %in% SELECTED_CELLTYPES
all_data_shared <- all_data_shared[, keep]
coldata_shared  <- coldata_shared[keep, ]
coldata_shared$celltype <- droplevels(coldata_shared$celltype)

message("Subset to ", ncol(all_data_shared), " samples across ",
        nlevels(coldata_shared$celltype), " cell types:")
message("  ", paste(levels(coldata_shared$celltype), collapse = ", "))

In [ ]:
# =============================================================================
# Cell 6: Filter peaks before DESeq2
# =============================================================================
# DESeq2 is O(n) per peak for dispersion estimation. 713K peaks is very
# slow (hours). We apply biologically motivated filters to focus on
# informative peaks for cross-species comparison:
#
# Filter 1: MIN_COUNT -- peak must have at least this many reads in at
#           least one sample. Removes low-count noise.
#
# Filter 2: MIN_SPECIES -- peak must have signal (>= 10 counts in at least
#           one sample) in at least MIN_SPECIES species. For cross-species
#           DESeq2, peaks that only appear in 1 species are uninformative.
#
# Adjust MIN_COUNT and MIN_SPECIES to control the trade-off between
# sensitivity and speed.

MIN_COUNT   <- 50   # min max-count across any sample
MIN_SPECIES <- 6    # require signal in all 6 species (most conservative)
                    # set to 2 for more permissive filtering

n_before <- nrow(all_data_shared)

# Filter 1: max count across any sample >= MIN_COUNT
max_per_peak <- apply(all_data_shared, 1, max)
keep_count <- max_per_peak >= MIN_COUNT

# Filter 2: signal in >= MIN_SPECIES species
n_species_with_signal <- rep(0L, nrow(all_data_shared))
for (sp in SPECIES) {
  sp_cols <- grep(paste0("^", sp, "_"), colnames(all_data_shared))
  if (length(sp_cols) > 0) {
    has_signal <- rowSums(all_data_shared[, sp_cols, drop = FALSE] >= 10) > 0
    n_species_with_signal <- n_species_with_signal + as.integer(has_signal)
  }
}
keep_species <- n_species_with_signal >= MIN_SPECIES

# Apply both filters
keep_peaks <- keep_count & keep_species
all_data_shared <- all_data_shared[keep_peaks, ]

message("Peak filtering:")
message("  Before:                        ", n_before)
message("  After max-count >= ", MIN_COUNT, ":       ", sum(keep_count))
message("  After signal in >= ", MIN_SPECIES, " species: ", sum(keep_peaks))
message("  Kept: ", sum(keep_peaks), " (", round(100 * sum(keep_peaks) / n_before, 1), "%)")
message("  Removed: ", n_before - sum(keep_peaks))

In [ ]:
# =============================================================================
# Cell 7: DESeq2 -- full model with species + celltype for PCA
# =============================================================================
# Design: ~ species + celltype
# This accounts for cell type differences when estimating species effects,
# and gives us a variance-stabilized matrix for PCA.

dds <- DESeqDataSetFromMatrix(
  countData = round(all_data_shared),
  colData   = coldata_shared,
  design    = ~ species + celltype
)

message("Running DESeq2 on ", nrow(all_data_shared), " peaks x ", ncol(all_data_shared), " samples...")
dds <- DESeq(dds)
message("Variance-stabilizing transform...")
vsd <- vst(dds)
message("Done")

In [ ]:
# =============================================================================
# Cell 7: PCA plot -- color by species, shape by cell type
# =============================================================================
pcaData <- plotPCA(vsd, intgroup = c("species", "celltype"), returnData = TRUE)
pct_var <- round(100 * attr(pcaData, "percentVar"))

species_colors <- c(
  Human = "#E41A1C", Bonobo = "#377EB8", Chimpanzee = "#4DAF4A",
  Gorilla = "#984EA3", Macaque = "#FF7F00", Marmoset = "#A65628"
)

p_pca <- ggplot(pcaData, aes(PC1, PC2, color = species, shape = celltype)) +
  geom_point(size = 3.5, alpha = 0.85) +
  scale_color_manual(values = species_colors) +
  labs(
    x = paste0("PC1 (", pct_var[1], "%)"),
    y = paste0("PC2 (", pct_var[2], "%)"),
    title = "PCA: Cross-species ATAC-seq accessibility",
    color = "Species", shape = "Cell type"
  ) +
  theme_minimal(base_size = 13) +
  theme(legend.position = "right")

ggsave(file.path(OUT_DIR, "plots/pca_species_celltype.pdf"), p_pca, width = 12, height = 7)
message("Saved PCA plot")
print(p_pca)

In [ ]:
# =============================================================================
# Cell 8: Pairwise DESeq2 -- Human vs each other species
# =============================================================================
# For each comparison, we subset to just Human + one other species,
# re-run DESeq2 with ~ species design, and save:
#   - Full results TSV
#   - BED files for human-gained and species-gained peaks
#   - Both at padj < 0.05 and at padj < 0.05 & |log2FC| > 1

# Load BED coordinates for output
bed_coords <- read_tsv(BED_FILE, col_names = c("chr", "start", "end", "region_id"),
                       show_col_types = FALSE)

species_to_compare <- setdiff(SPECIES, "Human")

for (sp in species_to_compare) {
  message("--- Running DESeq2: Human vs ", sp, " ---")

  # Subset samples to Human + this species
  dds_sub <- dds[, dds$species %in% c("Human", sp)]
  dds_sub$species <- droplevels(dds_sub$species)
  design(dds_sub) <- ~ species

  # Run DESeq2
  dds_sub <- DESeq(dds_sub, quiet = TRUE)

  # Extract results (positive LFC = higher in Human)
  res <- results(dds_sub, contrast = c("species", "Human", sp)) %>%
    as.data.frame() %>%
    rownames_to_column("region_id")

  # Save full results
  write_tsv(res, file.path(OUT_DIR, paste0("DESeq2_Human_vs_", sp, ".tsv")))

  # Significant peaks (padj < 0.05)
  sig <- res %>% filter(!is.na(padj) & padj < 0.05)
  sig_coords <- left_join(sig, bed_coords, by = "region_id")

  # Human-gained (LFC > 0) and species-gained (LFC < 0)
  human_up   <- sig_coords %>% filter(log2FoldChange > 0) %>% select(chr, start, end)
  species_up <- sig_coords %>% filter(log2FoldChange < 0) %>% select(chr, start, end)

  write_tsv(human_up,   file.path(OUT_DIR, paste0("Human_vs_", sp, "_HumanSpecific.bed")), col_names = FALSE)
  write_tsv(species_up, file.path(OUT_DIR, paste0("Human_vs_", sp, "_", sp, "Specific.bed")), col_names = FALSE)

  # Stringent filter: |log2FC| > 1
  human_strong   <- sig_coords %>% filter(log2FoldChange > 1)  %>% select(chr, start, end)
  species_strong <- sig_coords %>% filter(log2FoldChange < -1) %>% select(chr, start, end)

  write_tsv(human_strong,   file.path(OUT_DIR, paste0("Human_vs_", sp, "_HumanSpecific_logFC1.bed")), col_names = FALSE)
  write_tsv(species_strong, file.path(OUT_DIR, paste0("Human_vs_", sp, "_", sp, "Specific_logFC1.bed")), col_names = FALSE)

  message("  Significant: ", nrow(sig),
          " | Human-gained: ", nrow(human_up), " (", nrow(human_strong), " at |LFC|>1)",
          " | ", sp, "-gained: ", nrow(species_up), " (", nrow(species_strong), " at |LFC|>1)")
}
message("\nAll pairwise comparisons done.")

In [ ]:
# =============================================================================
# Cell 9: Heatmap -- top variable peaks across all species
# =============================================================================
# Use the variance-stabilized data for the heatmap.
# Select the most variable peaks and cluster both rows and columns.

# Get normalized counts from VST
vsd_mat <- assay(vsd)

# Select top 500 most variable peaks
rv <- rowVars(vsd_mat)
top_n <- min(500, sum(rv > 0))
top_idx <- order(rv, decreasing = TRUE)[1:top_n]
heatmap_mat <- vsd_mat[top_idx, ]

# Z-score each peak (row) for visualization
heatmap_z <- t(scale(t(heatmap_mat)))
heatmap_z[heatmap_z > 3]  <-  3
heatmap_z[heatmap_z < -3] <- -3

# Annotation bars for columns
ann_col <- data.frame(
  Species  = coldata_shared[colnames(heatmap_z), "species"],
  CellType = coldata_shared[colnames(heatmap_z), "celltype"],
  row.names = colnames(heatmap_z)
)

ann_colors <- list(
  Species = species_colors[levels(coldata_shared$species)]
)

pdf(file.path(OUT_DIR, "plots/heatmap_top500_variable.pdf"), width = 14, height = 10)
pheatmap(
  heatmap_z,
  annotation_col = ann_col,
  annotation_colors = ann_colors,
  cluster_rows = TRUE,
  cluster_cols = TRUE,
  show_rownames = FALSE,
  show_colnames = TRUE,
  fontsize_col = 6,
  color = colorRampPalette(rev(brewer.pal(11, "RdBu")))(100),
  main = "Top 500 Variable Peaks (Z-scored VST values)"
)
dev.off()
message("Saved heatmap to ", file.path(OUT_DIR, "plots/heatmap_top500_variable.pdf"))

# Also display inline
pheatmap(
  heatmap_z,
  annotation_col = ann_col,
  annotation_colors = ann_colors,
  cluster_rows = TRUE,
  cluster_cols = TRUE,
  show_rownames = FALSE,
  show_colnames = TRUE,
  fontsize_col = 6,
  color = colorRampPalette(rev(brewer.pal(11, "RdBu")))(100),
  main = "Top 500 Variable Peaks (Z-scored VST values)"
)

In [ ]:
# =============================================================================
# Cell 10: Sample-to-sample correlation heatmap
# =============================================================================
sample_cor <- cor(vsd_mat, method = "pearson")

pdf(file.path(OUT_DIR, "plots/heatmap_sample_correlation.pdf"), width = 13, height = 11)
pheatmap(
  sample_cor,
  annotation_col = ann_col,
  annotation_row = ann_col,
  annotation_colors = ann_colors,
  color = colorRampPalette(brewer.pal(9, "YlOrRd"))(100),
  fontsize_row = 5,
  fontsize_col = 5,
  main = "Sample-to-Sample Pearson Correlation"
)
dev.off()
message("Saved correlation heatmap")

pheatmap(
  sample_cor,
  annotation_col = ann_col,
  annotation_row = ann_col,
  annotation_colors = ann_colors,
  color = colorRampPalette(brewer.pal(9, "YlOrRd"))(100),
  fontsize_row = 5,
  fontsize_col = 5,
  main = "Sample-to-Sample Pearson Correlation"
)

In [ ]:
# =============================================================================
# Cell 11: Manhattan plot function
# =============================================================================
# Plots -log10(pvalue) across genomic position, alternating colors by chromosome.

plot_manhattan <- function(deseq_file, bed_file, chromsizes_file,
                           output_file, plot_title = NULL,
                           width = 16, height = 5) {

  deseq_res  <- read_tsv(deseq_file, show_col_types = FALSE)
  bed_coords <- read_tsv(bed_file, col_names = c("chr", "start", "end", "region_id"),
                         show_col_types = FALSE)
  chromsizes <- read_tsv(chromsizes_file, col_names = c("chr", "size"),
                         show_col_types = FALSE)

  # Order chromosomes naturally and compute genomic offsets
  chromsizes <- chromsizes %>%
    filter(chr %in% bed_coords$chr) %>%
    arrange(mixedorder(chr)) %>%
    mutate(offset = lag(cumsum(size), default = 0))

  # Join DESeq2 results with coordinates
  plot_data <- left_join(deseq_res, bed_coords, by = "region_id") %>%
    filter(!is.na(chr)) %>%
    left_join(chromsizes, by = "chr") %>%
    mutate(genomic_pos = offset + (start + end) / 2)

  # Chromosome midpoints for axis labels
  chr_ticks <- chromsizes %>% mutate(midpoint = offset + size / 2)

  if (is.null(plot_title)) {
    plot_title <- tools::file_path_sans_ext(basename(deseq_file))
  }

  p <- ggplot(plot_data, aes(x = genomic_pos, y = -log10(pvalue), color = chr)) +
    geom_point(size = 0.5, alpha = 0.8, show.legend = FALSE) +
    scale_color_manual(values = rep(c("#1f77b4", "#ff7f0e"), length.out = nrow(chr_ticks))) +
    scale_x_continuous(
      label = chr_ticks$chr,
      breaks = chr_ticks$midpoint,
      expand = expansion(mult = c(0.01, 0.01))
    ) +
    labs(x = "Chromosome", y = expression(-log[10](pvalue)), title = plot_title) +
    theme_minimal(base_size = 14) +
    theme(
      axis.text.x = element_text(angle = 90, vjust = 0.5, size = 10),
      panel.grid.major.x = element_blank(),
      plot.title = element_text(hjust = 0.5)
    )

  ggsave(output_file, p, width = width, height = height)
  message("Saved: ", output_file)
  return(p)
}

message("Manhattan plot function defined")

In [ ]:
# =============================================================================
# Cell 12: Generate Manhattan plots for all pairwise comparisons
# =============================================================================

for (sp in species_to_compare) {
  deseq_file  <- file.path(OUT_DIR, paste0("DESeq2_Human_vs_", sp, ".tsv"))
  output_file <- file.path(OUT_DIR, "plots", paste0("Manhattan_Human_vs_", sp, ".pdf"))

  p <- plot_manhattan(
    deseq_file     = deseq_file,
    bed_file       = BED_FILE,
    chromsizes_file = CHROMSIZES_FILE,
    output_file    = output_file,
    plot_title     = paste0("Human vs ", sp)
  )
  print(p)
}

In [ ]:
# =============================================================================
# Cell 13: Summary of results
# =============================================================================
message("=== Analysis Summary ===")
message("Peaks: ", nrow(all_data_shared))
message("Samples: ", ncol(all_data_shared))
message("Shared cell types: ", paste(sort(shared_ct), collapse = ", "))
message("")

# Summarize each comparison
for (sp in species_to_compare) {
  res_file <- file.path(OUT_DIR, paste0("DESeq2_Human_vs_", sp, ".tsv"))
  if (file.exists(res_file)) {
    res <- read_tsv(res_file, show_col_types = FALSE)
    sig <- res %>% filter(!is.na(padj) & padj < 0.05)
    human_up   <- sum(sig$log2FoldChange > 0, na.rm = TRUE)
    human_down <- sum(sig$log2FoldChange < 0, na.rm = TRUE)
    message(sprintf("Human vs %-12s  sig: %6d  human-gained: %5d  %s-gained: %5d",
                    sp, nrow(sig), human_up, sp, human_down))
  }
}

message("\nOutput files:")
for (f in sort(list.files(OUT_DIR, recursive = TRUE))) {
  fsize <- file.size(file.path(OUT_DIR, f))
  message(sprintf("  %-55s %s", f,
          ifelse(fsize > 1e6, paste0(round(fsize/1e6, 1), " MB"),
                              paste0(round(fsize/1e3), " KB"))))
}